In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Media_Bias_Feature_for_the_Identification_of_Biased_Statements_in_News_Articles')
!pwd

/content
/content/drive/MyDrive/Colab Notebooks/Media_Bias_Feature_for_the_Identification_of_Biased_Statements_in_News_Articles


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
xl_file = pd.ExcelFile("Datasets/final_labels_SG2.xlsx")

df_load = pd.read_excel(xl_file, sheet_name='Sheet1')

df_load.head()

,text,news_link,outlet,topic,type,label_bias,label_opinion,biased_words
0,"""Orange Is the New Black"" star Yael Stone is r...",https://www.foxnews.com/entertainment/australi...,Fox News,environment,right,Non-biased,Entirely factual,[]
1,"""We have one beautiful law,"" Trump recently sa...",https://www.alternet.org/2020/06/law-and-order...,Alternet,gun control,left,Biased,Somewhat factual but also opinionated,"['bizarre', 'characteristically']"
2,"...immigrants as criminals and eugenics, all o...",https://www.nbcnews.com/news/latino/after-step...,MSNBC,white-nationalism,left,Biased,Expresses writer’s opinion,"['criminals', 'fringe', 'extreme']"
3,...we sounded the alarm in the early months of...,https://www.alternet.org/2019/07/fox-news-has-...,Alternet,white-nationalism,left,Biased,Somewhat factual but also opinionated,[]
4,[Black Lives Matter] is essentially a non-fals...,http://feedproxy.google.com/~r/breitbart/~3/-v...,Breitbart,marriage-equality,NaN,Biased,Expresses writer’s opinion,['cult']


In [ ]:
df_load['label_opinion'].value_counts()

Entirely factual                         1600
Somewhat factual but also opinionated    1000
Expresses writer’s opinion                858
No agreement                              216
Name: label_opinion, dtype: int64

In [ ]:
df_load.drop(df_load[df_load.label_opinion == "No agreement"].index, inplace = True)

In [ ]:
df_load['label_opinion'].value_counts()

Entirely factual                         1600
Somewhat factual but also opinionated    1000
Expresses writer’s opinion                858
Name: label_opinion, dtype: int64

In [ ]:
df = df_load.loc[:, ['text', 'label_opinion']]

In [ ]:
df.head()

,text,label_opinion
0,"""Orange Is the New Black"" star Yael Stone is r...",Entirely factual
1,"""We have one beautiful law,"" Trump recently sa...",Somewhat factual but also opinionated
2,"...immigrants as criminals and eugenics, all o...",Expresses writer’s opinion
3,...we sounded the alarm in the early months of...,Somewhat factual but also opinionated
4,[Black Lives Matter] is essentially a non-fals...,Expresses writer’s opinion


In [ ]:
df['objective'] = ''
df['partly-subjective'] = ''
df['subjective'] = ''

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3458 entries, 0 to 3673
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               3458 non-null   object
 1   label_opinion      3458 non-null   object
 2   objective          3458 non-null   object
 3   partly-subjective  3458 non-null   object
 4   subjective         3458 non-null   object
dtypes: object(5)
memory usage: 162.1+ KB


In [ ]:
def label_subjectivity (row):
   if row['label_opinion'] == "Entirely factual" :
      row['objective'] = 1
      row['partly-subjective'] = 0
      row['subjective'] = 0
      return 0
   elif row['label_opinion'] == "Somewhat factual but also opinionated"  :
      row['objective'] = 0
      row['partly-subjective'] = 1
      row['subjective'] = 0
      return 1
   elif row['label_opinion'] == "Expresses writer’s opinion" :
      row['objective'] = 0
      row['partly-subjective'] = 0
      row['subjective'] = 1
      return 2
   else:
      return None

In [ ]:
df['label_opinion'] = df.apply (lambda row: label_subjectivity(row), axis=1)

In [ ]:
df.head()

,text,label_opinion,objective,partly-subjective,subjective
0,"""Orange Is the New Black"" star Yael Stone is r...",0,1,0,0
1,"""We have one beautiful law,"" Trump recently sa...",1,0,1,0
2,"...immigrants as criminals and eugenics, all o...",2,0,0,1
3,...we sounded the alarm in the early months of...,1,0,1,0
4,[Black Lives Matter] is essentially a non-fals...,2,0,0,1


In [ ]:
train_df_biased_statements = pd.read_csv("Datasets/Splits/Train/final_labels_SG2_label_bias_train.csv")
test_df_biased_statements = pd.read_csv("Datasets/Splits/Test/final_labels_SG2_label_bias_test.csv")

In [ ]:
train_df = train_df_biased_statements.merge(df, how='inner', on='text')
train_df = train_df.drop(['bias'], axis=1)
train_df = train_df.dropna()

In [ ]:
train_df_biased_statements

,text,bias
0,Letters from GOP members of Congress and natio...,1
1,Activist Dorian Wilde said anti-LGBT statement...,0
2,U.S. President Donald Trump retweeted a video ...,1
3,Everything in the progressive agenda is aimed ...,1
4,Even pro-life leaders have expressed their app...,0
...,...,...
2566,Gun enthusiasts cannot admit that they like fi...,0
2567,"In 2016, Donald Trump fueled his improbable pr...",1
2568,Doctors say over-usage of the body in competit...,0
2569,Well-intentioned politicians might want to pro...,0


In [ ]:
train_df

,text,label_opinion,objective,partly-subjective,subjective
0,Letters from GOP members of Congress and natio...,1,0,1,0
1,Activist Dorian Wilde said anti-LGBT statement...,0,1,0,0
2,U.S. President Donald Trump retweeted a video ...,0,1,0,0
3,Everything in the progressive agenda is aimed ...,2,0,0,1
4,Even pro-life leaders have expressed their app...,0,1,0,0
...,...,...,...,...,...
2398,Gun enthusiasts cannot admit that they like fi...,2,0,0,1
2399,"In 2016, Donald Trump fueled his improbable pr...",1,0,1,0
2400,Doctors say over-usage of the body in competit...,0,1,0,0
2401,Well-intentioned politicians might want to pro...,2,0,0,1


In [ ]:
test_df = test_df_biased_statements.merge(df, how='inner', on='text')
test_df = test_df.drop(['bias'], axis=1)
test_df = test_df.dropna()

In [ ]:
test_df_biased_statements

,text,bias
0,Protests erupted in Louisville and several oth...,0
1,The legislation is the most sweeping college a...,0
2,Left-wing Christians declare that the real way...,1
3,Perhaps the pushback from the Pentagon will in...,1
4,Trump's latest conspiracy theory seems to echo...,0
...,...,...
1097,Thousands of asylum seekers crammed in border ...,1
1098,U.S. President Donald Trump is open to negotia...,0
1099,The researchers analyzed data collected betwee...,0
1100,Local health officer Dr Rosana Salvaterra appe...,1


In [ ]:
test_df

,text,label_opinion,objective,partly-subjective,subjective
0,Protests erupted in Louisville and several oth...,0,1,0,0
1,The legislation is the most sweeping college a...,0,1,0,0
2,Left-wing Christians declare that the real way...,0,1,0,0
3,Perhaps the pushback from the Pentagon will in...,2,0,0,1
4,"Ivanka Trump, President Trump’s daughter and a...",0,1,0,0
...,...,...,...,...,...
1049,Thousands of asylum seekers crammed in border ...,0,1,0,0
1050,U.S. President Donald Trump is open to negotia...,0,1,0,0
1051,The researchers analyzed data collected betwee...,0,1,0,0
1052,Local health officer Dr Rosana Salvaterra appe...,1,0,1,0


In [ ]:
train_df.shape, test_df.shape

((2403, 5), (1054, 5))

In [ ]:
train_df.to_csv('Datasets/Splits/Train/final_labels_SG2_label_subjectivity_train.csv', index = False)

In [ ]:
test_df.to_csv('Datasets/Splits/Test/final_labels_SG2_label_subjectivity_test.csv', index = False)